In [0]:
%run ./aws_postgres_keys

### Postgres jdbc URL

In [0]:
jdbcURL = "jdbc:postgresql://{}/{}".format(HOST,DATABASE_NAME)

### Create connection properties with Username & password

In [0]:
connProperties = {
        'user':USER_NAME,
        'password':PASSWORD
}

### Read the table from aws_postgres_sql

In [0]:
df_contrato = spark.read.jdbc(url=jdbcURL,table='contrato',properties = connProperties)
df_cliente = spark.read.jdbc(url=jdbcURL,table='clientes',properties = connProperties)
df_rating = spark.read.jdbc(url=jdbcURL,table='rating',properties = connProperties)
df_rating_cliente = spark.read.jdbc(url=jdbcURL,table='rating_cliente',properties = connProperties)

In [0]:
display(df_contrato)



In [0]:
display(df_cliente)


In [0]:
display(df_rating)


In [0]:
display(df_rating_cliente)

### Criar tabela Bronze

In [0]:
df = df.toDF(*[col.replace('.', '_') for col in df.columns])
display(df)

In [0]:
df_contrato.write.mode("overwrite").saveAsTable("`impacta-databricks`.bronze.contrato")
df_cliente.write.mode("overwrite").saveAsTable("`impacta-databricks`.bronze.cliente")
df_rating.write.mode("overwrite").saveAsTable("`impacta-databricks`.bronze.rating")
df_rating_cliente.write.mode("overwrite").saveAsTable("`impacta-databricks`.bronze.rating_cliente")

In [0]:
df_bronze_contrato = spark.table("`impacta-databricks`.bronze.contrato")
df_bronze_cliente = spark.table("`impacta-databricks`.bronze.cliente")
df_bronze_rating = spark.table("`impacta-databricks`.bronze.rating")
df_bronze_rating_cliente = spark.table("`impacta-databricks`.bronze.rating_cliente")


In [0]:
display(df_bronze_contrato)

In [0]:
display(df_bronze_cliente)

In [0]:
display(df_bronze_rating)

In [0]:
display(df_bronze_rating_cliente)